In [41]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
      pip install langchain sentence-transformers chromadb llama-cpp-python langchain-community pypdf

#Importing libraries


In [43]:
#to load the pdf file and extract the text
from langchain_community.document_loaders import PyPDFDirectoryLoader
#It splits the text into chuncks
from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
#To generate embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
#To store the embeddings with chuncks in chroma
from langchain.vectorstores import Chroma
#To load the LLM model
from langchain_community.llms import LlamaCpp
#To develop the model End to End
from langchain.chains import RetrievalQA, LLMChain

#import documents


In [44]:
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/medical chatbot/")
docs = loader.load()

In [45]:
len(docs) #number of pages

95

In [46]:
docs[5] #It will show the content of page 5.

Document(metadata={'source': '/content/drive/MyDrive/medical chatbot/healthyheart.pdf', 'page': 5}, page_content='If you’re like many people, you may think of heart disease as a\nproblem that happens to other folks. “I feel fine,” you may think,\n“so I have nothing to worry about.” If you’re a woman, you mayalso believe that being female protects you from heart disease.If you’re a man, you may think you’re not old enough to have aserious heart condition.\nWrong on all counts. In the United States, heart disease is the #1\nkiller of both women and men. It affects many people at midlife, as well as in old age. It also can happen to those who “feel fine.”Consider these facts: \n■Each year, 500,000 Americans die of heart disease, and approx-imately half of them are women.\n■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, risk starts to increase at age 55.\n■Fifty percent of men and 64 percent of women who die suddenlyof heart disease have no pre

#chunking


In [47]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = text_splitter.split_documents(docs)


In [48]:
len(chunks)

747

In [49]:
chunks[150]

Document(metadata={'source': '/content/drive/MyDrive/medical chatbot/healthyheart.pdf', 'page': 18}, page_content='Be aware, too, that a high systolic blood pressure level (first number)')

Embedding creation

In [50]:
import os
os.environ["HuggingFacehub-API-Token"]="Hugging face access token"


In [ ]:
embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")

##vector store creation


In [52]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [73]:
query="who is at the risk of heart disease"
searchresult=vectorstore.similarity_search(query)

In [74]:
searchresult

[Document(metadata={'page': 8, 'source': '/content/drive/MyDrive/medical chatbot/healthyheart.pdf'}, page_content='While each risk factor increases your risk of heart disease, having'),
 Document(metadata={'page': 8, 'source': '/content/drive/MyDrive/medical chatbot/healthyheart.pdf'}, page_content='While each risk factor increases your risk of heart disease, having'),
 Document(metadata={'page': 5, 'source': '/content/drive/MyDrive/medical chatbot/healthyheart.pdf'}, page_content='■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, risk starts to increase at age 55.\n■Fifty percent of men and 64 percent of women who die suddenlyof heart disease have no previous symptoms of the disease.1Heart Disease: Why Should You Care?Heart Disease:'),
 Document(metadata={'page': 5, 'source': '/content/drive/MyDrive/medical chatbot/healthyheart.pdf'}, page_content='■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, 

#Instead of using search result we can build a retriever

In [75]:
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 5}
)
retriever.get_relevant_documents(query)

[Document(metadata={'page': 8, 'source': '/content/drive/MyDrive/medical chatbot/healthyheart.pdf'}, page_content='While each risk factor increases your risk of heart disease, having'),
 Document(metadata={'page': 8, 'source': '/content/drive/MyDrive/medical chatbot/healthyheart.pdf'}, page_content='While each risk factor increases your risk of heart disease, having'),
 Document(metadata={'page': 5, 'source': '/content/drive/MyDrive/medical chatbot/healthyheart.pdf'}, page_content='■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, risk starts to increase at age 55.\n■Fifty percent of men and 64 percent of women who die suddenlyof heart disease have no previous symptoms of the disease.1Heart Disease: Why Should You Care?Heart Disease:'),
 Document(metadata={'page': 5, 'source': '/content/drive/MyDrive/medical chatbot/healthyheart.pdf'}, page_content='■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, 

#LLM loading

In [56]:
llm = LlamaCpp(
    model_path= "/content/drive/MyDrive/medical chatbot/BioMistral-7B-GGUF/BioMistral-7B.Q4_K_S.gguf",
    temperature=0.3,
    max_tokens=2048,
    top_p=1)

llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/drive/MyDrive/medical chatbot/BioMistral-7B-GGUF/BioMistral-7B.Q4_K_S.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:  

#Use LLM, Retriever and Query to generate final response

In [67]:
template = """
<|context|> #retriver context will be here
You are an AI assistant that follows instruction extremely well.
Please be truthful and give direct answers
</s>
<|user|>
{query} #user query will be here
</s>
 <|assistant|> #Role of llm
"""

#To build the chain connection use langchain

In [68]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

In [69]:
prompt = ChatPromptTemplate.from_template(template)

In [70]:
rag_chain = (
    {"context": retriever,  "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#calling the rag chain to display output

In [71]:
response = rag_chain.invoke(query)

Llama.generate: 21 prefix-match hit, remaining 57 prompt tokens to eval

llama_print_timings:        load time =    3427.38 ms
llama_print_timings:      sample time =       9.02 ms /    13 runs   (    0.69 ms per token,  1441.40 tokens per second)
llama_print_timings: prompt eval time =   27867.13 ms /    56 tokens (  497.63 ms per token,     2.01 tokens per second)
llama_print_timings:        eval time =    9410.54 ms /    13 runs   (  723.89 ms per token,     1.38 tokens per second)
llama_print_timings:       total time =   37300.32 ms /    69 tokens


In [76]:
response

'<|context|> #retriver context will be here'

#To make a interactive chat

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = rag_chain.invoke(user_input)
  print("Answer: ",result)

Input Prompt: what is heart disease


Llama.generate: 48 prefix-match hit, remaining 26 prompt tokens to eval

llama_print_timings:        load time =    3427.38 ms
llama_print_timings:      sample time =      27.75 ms /    40 runs   (    0.69 ms per token,  1441.29 tokens per second)
llama_print_timings: prompt eval time =   13071.68 ms /    26 tokens (  502.76 ms per token,     1.99 tokens per second)
llama_print_timings:        eval time =   26476.76 ms /    39 runs   (  678.89 ms per token,     1.47 tokens per second)
llama_print_timings:       total time =   39614.58 ms /    65 tokens


Answer:  I am a large language model, I can tell you about the statistics of heart diseases in different countries. What country would you like to know more about? #response from llm will be here
